In [42]:
import os
import openai
import pandas as pd
import pdfplumber
import json
import numpy as np

import config

In [43]:
from openpyxl import load_workbook

file_path = './truth_excel/test_all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [44]:
import openai
from openai import OpenAI
client = OpenAI()

In [45]:
def get_file_names(directory):
    """Gets a list of file names in the specified directory.

    Args:
        directory (str): The path to the directory.

    Returns:
        list: A list of file names.
    """

    file_names = []
    for entry in os.scandir(directory):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

directory_path = "../Desktop/Testing"
file_names = get_file_names(directory_path)
print(file_names)

['rynn2008.pdf', 'lennox2003.pdf', 'kasper2014.pdf', 'hartford2007.pdf', 'boyer2004.pdf', 'merideth2012.pdf', 'mahablesh2013.pdf', 'davidson2004.pdf', 'pollock2001.pdf', 'nicolini2009.pdf', 'allgulander2004.pdf', 'pollock2008a.pdf', 'wu2011.pdf', 'bose2008.pdf', 'rickels2003.pdf', 'rothschild2012.pdf', 'stein2008.pdf', 'khan2011.pdf', 'alaka2014.pdf', 'ball2015.pdf', 'nimatoudis2004.pdf']


# Pollock2008a was omitted due to heavy data discrepencies 

In [46]:
from pdf2image import convert_from_path
import pdfplumber
import pytesseract
import pandas as pd

# Function to extract text and tables
def extract_pdf_content(file_path):
    text = ""
    tables = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text += page.extract_text() + "\n"
            
            # Extract tables
            for table in page.extract_tables():
                tables.append(pd.DataFrame(table))
    return text, tables

# Function to extract images
def extract_images_from_pdf(file_path):
    images = convert_from_path(file_path)
    return images

# Function to process images using OCR
def process_images_with_ocr(images):
    ocr_results = []
    for img in images:
        text = pytesseract.image_to_string(img)
        ocr_results.append(text)
    return ocr_results

In [74]:
from pdf2image import convert_from_path
import base64
from io import BytesIO
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

def convert_pdf_to_base64_images(pdf_path, dpi=200, image_format='JPEG'):
    images = convert_from_path(pdf_path, dpi=dpi)
    base64_images = []
    for img in images:
        buffer = BytesIO()
        img.save(buffer, format=image_format)
        buffer.seek(0)
        image_data = buffer.getvalue()
        image_base64 = base64.b64encode(image_data).decode('utf-8')
        base64_images.append(image_base64)
    return base64_images

def query_gpt4_full(text, tables, pdf_path):
    # Convert all PDF pages to base64-encoded images
    base64_images = convert_pdf_to_base64_images(pdf_path)
    
    structured_prompt_text = (
        f"Clinical Trial Report Analysis:\n\n"
        f"Extracted Text:\n{text}\n\n"
        f"Extracted Tables:\n{tables}\n\n"
        f"This is a clinical trial report. For EACH intervention in the trial (including placebo), "
        f"please extract the following characteristics and format the response as valid JSON using this exact example structure:\n\n"
        f"Example format:\n"
        f"{{\n"
        f'    "Last Name of Main Author and Year": "Doe et al., 2021",\n'
        f'    "Full Population Sample Size": "451",\n'
        f'    "Intervention": "Duloxetine: 50 mg/day",\n'
        f'    "Main Race": "White",\n'
        f'    "Percent of Intervention Population that is Female (%)": "61.5",\n'
        f'    "Mean HAMA Score": "24.5",\n'
        f'    "Mean Population Age (Year)": "43.2",\n'
        f'    "Attrition Rate (%)": "30.2",\n'
        f'    "Full Sponsor Name": "ABC Pharmaceuticals",\n'
        f'    "Follow-up Time (Weeks)": "10",\n'
        f'    "Diagnostic Criteria": "DSM-IV"\n'
        f"}}\n\n"
        f"'Full Population Sample Size' should refer to the TOTAL population enrolled in the study, across all interventions and groups, not just the population size for the specific intervention.\n"
        f"'Intervention' should be in mg/day, not any other unit of measurement.\n"
        f"'Follow-up Time' should refer to total length of the treatment period, omitting washout periods.\n"
        f"'Mean HAMA' should be the mean HAMA score at the beginning of the study for the specific intervention.\n"
        f"'Attrition Rate' should be % of patients who failed to complete the treatment after assignment\n"
        f" Make sure each JSON object follows this format exactly. For any missing or unavailable data, input 'NA'. If you are unsure about an answer, input 'NA'."
    )

    # Construct the message content as a list: first the text, then the images
    message_content = []
    message_content.append({
        "type": "text",
        "text": structured_prompt_text
    })

    # Add each page of the PDF as an image message
    for img_b64 in base64_images:
        message_content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{img_b64}"
            }
        })

    # Create the chat completion request
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a data extraction assistant extracting data from clinical trial reports."},
            {"role": "user", "content": message_content}
        ],
    )

    return response

In [47]:
# def query_gpt4_text(text, tables, ocr_results):
#     structured_prompt = (
#         f"Clinical Trial Report Analysis:\n\n"
#         f"Extracted Text:\n{text}\n\n"
#         f"Extracted OCR Text from Images:\n{ocr_results}\n\n"
#         f"Extracted Tables:\n{tables}\n\n"
#         f"This is a clinical trial report. For EACH intervention in the trial (including placebo), please extract the following characteristics and format the response as valid JSON using this exact example structure:\n\n"
#         f"Example format:\n"
#         f"{{\n"
#         f'    "Last Name of Main Author and Year": "Doe et al., 2021",\n'
#         f'    "Full Population Sample Size": "451",\n'
#         f'    "Intervention": "Duloxetine: 50 mg/day",\n'
#         f'    "Main Race": "White",\n'
#         f'    "Percent of Intervention Population that is Female (%)": "61.5",\n'
#         f'    "Mean HAMA Score": "24.5",\n'
#         f'    "Mean Population Age (Year)": "43.2",\n'
#         f'    "Attrition Rate (%)": "30.2",\n'
#         f'    "Full Sponsor Name": "ABC Pharmaceuticals",\n'
#         f'    "Follow-up Time (Weeks)": "10",\n'
#         f'    "Diagnostic Criteria": "DSM-IV"\n'
#         f"}}\n\n"
#         f"'Full Population Sample Size' should refer to the TOTAL population enrolled in the study, across all interventions and groups, not just the population size for the specific intervention.\n"
#         f"'Intervention' should be in mg/day, not any other unit of measurement.\n"
#         f"'Follow-up Time' should refer to total length of the treatment period, omitting washout periods.\n"
#         f"'Mean HAMA' should be the mean HAMA score at the beginning of the study for the specific intervention.\n"
#         f"'Attrition Rate' should be % of patients who failed to complete the treatment after assignment\n"
#         f" Make sure each JSON object follows this format exactly. For any missing or unavailable data, input 'NA'. If you are unsure about an answer, input 'NA'."
#     )
#     response = completion = client.chat.completions.create(
#         model="gpt-4o",
#         messages=[{"role": "system", "content": "You are a data extraction assistant extracting data from clinical trial reports."},
#                   {"role": "user", "content": structured_prompt}],
#     )
#     return response

In [48]:
def rename_columns(df):
    import pandas as pd
    
    # Rename columns in df to match df_truth
    df = df.rename(columns={
        "Last Name of Main Author and Year": "References",
        "Full Population Sample Size": "Sample size",
        "Main Race": "Main race",
        "Intervention": "Interventions",
        "Percent of Study Population that is Female (%)": "Female (%)",
        "Percent of Intervention Population that is Female (%)": "Female (%)",
        "Mean HAMA Score": "Mean HAMA",
        "Mean Population Age (Year)": "Mean age (Year)",
        "Attrition Rate (%)": "Attrition rate (%)",
        "Full Sponsor Name": "Sponsor",
        "Follow-up Time (Weeks)": "Follow-up time (weeks)",
        "Diagnostic Criteria": "Diagnosis criteria"
    })
    return df

In [49]:
def preprocess_numerical_columns(df, numerical_cols):
    """
    Converts specified numerical columns to numeric types in a DataFrame.
    Keeps 'NA' as a string and does not convert it to NaN.
    """
    for col in numerical_cols:
        if col in df.columns:
            # Preserve 'NA' and convert the rest to numeric
            df[col] = df[col].apply(lambda x: x if str(x).strip().lower() == "na" else pd.to_numeric(x, errors="coerce"))
    return df

# Update the numerical columns list
numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]


In [50]:
def preprocess_interventions_and_diag(df):
    """
    Normalizes the 'Interventions' column in the DataFrame for consistent filtering.
    Replaces all dashes (e.g., en dash, em dash) with a standard hyphen.
    """
    df = df.copy()
    df["Interventions"] = (
        df["Interventions"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
        .str.replace(r":(?=\d)", ": ", regex=True)
        .str.replace(r"SR:", ":", regex = True)
        .str.replace(r"XR:", ":", regex = True)

    )
    df["Diagnosis criteria"] = (
    df["Diagnosis criteria"]
        .str.strip()
        .str.lower()
        .str.replace("–", "-", regex=False)  # Replace en dash with hyphen
        .str.replace("—", "-", regex=False)  # Replace em dash with hyphen
        .str.replace("\u00a0", " ")  # Replace non-breaking space with regular space
    )
    return df

In [51]:
def testing(df_truth, df):
    grouped_truth = df_truth.groupby("References")
    grouped_pred = df.groupby("References")
    
    results = []
    
    for reference in grouped_truth.groups:
        if reference in grouped_pred.groups:
            # Get groups
            truth_group = grouped_truth.get_group(reference)
            pred_group = grouped_pred.get_group(reference)
    
            # Normalize and find common interventions
            truth_interventions = set(truth_group["Interventions"].tolist())
            pred_interventions = set(pred_group["Interventions"].tolist())
            common_interventions = truth_interventions & pred_interventions  # Only common interventions
    
            for intervention in common_interventions:
                # Filter for the specific intervention
                truth_row = truth_group[truth_group["Interventions"] == intervention]
                pred_row = pred_group[pred_group["Interventions"] == intervention]
    
                if not truth_row.empty and not pred_row.empty:
                    truth_row = truth_row.iloc[0]
                    pred_row = pred_row.iloc[0]
    
                    # Categorical columns
                    categorical_cols = ["Main race"]
                    categorical_match = {
                        col: truth_row[col].strip().lower() == pred_row[col].strip().lower()
                        for col in categorical_cols
                    }
    
                    # Special handling for "Diagnosis criteria"
                    diagnosis_criteria_truth = truth_row["Diagnosis criteria"].strip().lower()
                    diagnosis_criteria_pred = pred_row["Diagnosis criteria"].strip().lower()
                    categorical_match["Diagnosis criteria"] = diagnosis_criteria_truth in diagnosis_criteria_pred

                    # Special handling for "Sponsor"
                    sponsor_truth = truth_row["Sponsor"].strip().lower()
                    sponsor_pred = pred_row["Sponsor"].strip().lower()
                    categorical_match["Sponsor"] = sponsor_truth in sponsor_pred

                    # Numerical columns
                    numerical_cols = ["Sample size", "Female (%)", "Mean HAMA", "Mean age (Year)", "Attrition rate (%)", "Follow-up time (weeks)"]
                    numerical_match = {}
                    for col in numerical_cols:
                        truth_val = truth_row[col]
                        pred_val = pred_row[col]
                        truth_is_na = str(truth_val).strip().lower() == "na"
                        pred_is_na = str(pred_val).strip().lower() == "na"

                        # Handle different scenarios for NA
                        if truth_is_na and pred_is_na:
                            numerical_match[col] = True  # Both are NA, match is True
                        elif truth_is_na or pred_is_na:
                            numerical_match[col] = False  # Only one is NA, match is False
                        else:
                            # Perform numerical comparison
                            numerical_match[col] = np.isclose(
                                float(truth_val), float(pred_val), atol=0.5, equal_nan=True
                            )
    
                    # Collect results
                    results.append({
                        "References": reference,
                        "Interventions": intervention,
                        **categorical_match,
                        **numerical_match,
                    })
    results_df = pd.DataFrame(results)
    return results_df

# Rynn 2008

In [75]:
file_path = "../Desktop/Testing/rynn2008.pdf"

# Extract content from PDF
rynn_2008_text, rynn_2008_tables = extract_pdf_content(file_path)

# Query GPT-4
structured_data_rynn_2008 = query_gpt4_full(rynn_2008_text, rynn_2008_tables, "../Desktop/Testing/rynn2008.pdf")